In [1]:
import pandas as pd

In [2]:
data_offline= pd.read_csv('ccf_offline.csv')
data_online = pd.read_csv('ccf_online (1).csv')

In [3]:
data_online.describe()

,User_id,Merchant_id,Action
count,1.142983e+07,1.142983e+07,1.142983e+07
mean,1.074683e+07,3.436686e+04,2.348193e-01
std,4.137712e+06,1.441243e+04,5.426314e-01
min,4.000000e+00,1.000100e+04,0.000000e+00
25%,1.019827e+07,2.090100e+04,0.000000e+00
50%,1.196970e+07,3.420000e+04,0.000000e+00
75%,1.373506e+07,4.741500e+04,0.000000e+00
max,1.550000e+07,6.000000e+04,2.000000e+00


In [4]:
data_offline.shape

(1754884, 7)

In [5]:
data_online.shape

(11429826, 7)

In [6]:
data_online[:5]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513,null
1,13740231,34805,1,null,null,null,20160321
2,14336199,18907,0,null,null,null,20160618
3,14336199,18907,0,null,null,null,20160618
4,14336199,18907,0,null,null,null,20160618


In [7]:
a=data_online[['Merchant_id','Coupon_id']]
mask = data_online['Coupon_id']!='null'


In [8]:
num_coupon = a[mask].groupby(['Merchant_id']).count().reset_index()


In [9]:
num_coupon.columns = ['Merchant_id','Tot_coupon_sent']

In [10]:
data_online1 = pd.merge(data_online, num_coupon, on = 'Merchant_id')

In [11]:
mask2 = (data_online1['Date']!= 'null') & (data_online1['Coupon_id']!= 'null')

In [12]:
b=data_online1[mask2].groupby(['Merchant_id'])['Action'].count().reset_index()
b.columns = ['Merchant_id','Num_coupon_used']

In [13]:
data_online2 = pd.merge(data_online1, b, on='Merchant_id')

In [14]:
l= data_online2[['Merchant_id','Coupon_id','Action']].groupby(['Merchant_id','Coupon_id']).count().reset_index()
mask3 = l['Coupon_id']!='null'
l2=l[mask3]


In [15]:
def num_type(s):
    if s>0:
        return 1

In [16]:
l2['num_type']=l2['Coupon_id'].apply(num_type)

C:\Users\zjj27\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
l3 =l2.groupby('Merchant_id')['num_type'].count()
l4=pd.DataFrame(l3).reset_index()
data_online3= pd.merge(data_online2, l4, on='Merchant_id')

In [18]:
data_online3.to_csv('data_online3.csv')

## For offline data

In [19]:
a=data_offline[['Merchant_id','Coupon_id']]
mask = data_offline['Coupon_id']!='null'
num_coupon = a[mask].groupby(['Merchant_id']).count().reset_index()
num_coupon.columns = ['Merchant_id','Tot_coupon_sent']
data_offline1 = pd.merge(data_offline, num_coupon, on = 'Merchant_id')

In [20]:
mask2 = (data_offline1['Date']!= 'null') & (data_offline1['Coupon_id']!= 'null')
b=data_offline1[mask2].groupby(['Merchant_id'])['Coupon_id'].count().reset_index()
b.columns = ['Merchant_id','Num_coupon_used']
data_offline2 = pd.merge(data_offline1, b, on='Merchant_id')
data_offline2[:5]

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Tot_coupon_sent,Num_coupon_used
0,1439408,2632,null,null,0,null,20160217,43,4
1,1439408,2632,8591,20:1,0,20160217,null,43,4
2,1439408,2632,1078,20:1,0,20160319,null,43,4
3,1439408,2632,8591,20:1,0,20160613,null,43,4
4,1439408,2632,null,null,0,null,20160516,43,4


In [21]:
l= data_offline2[['Merchant_id','Coupon_id']].groupby(['Merchant_id','Coupon_id']).count().reset_index()
mask3 = l['Coupon_id']!='null'
l2=l[mask3]
l2['num_type']=l2['Coupon_id'].apply(num_type)
l3 =l2.groupby('Merchant_id')['num_type'].count()
l4=pd.DataFrame(l3).reset_index()
data_offline3= pd.merge(data_offline2, l4, on='Merchant_id')

C:\Users\zjj27\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [22]:
null_mask = data_offline3['Distance']!='null'

def to_float(s):
    return float(s)

data_offline4 =data_offline3[null_mask]
data_offline4['Distance']=data_offline4['Distance'].map(to_float)
data_offline5 = pd.DataFrame(data_offline4.groupby('Merchant_id')['Distance'].mean()).reset_index()
data_offline6 = pd.merge(data_offline3, data_offline5,on='Merchant_id')

C:\Users\zjj27\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [23]:
data_offline6=data_offline6.rename(columns = {'Distance_y':'Avg_Distance'})

In [24]:
data_offline6.to_csv('data_offline6.csv')